# Entity Framework (ORM)

Uno de los principales problemas problemas que nos encontramos a la hora de comenzar a trabajar en un lenguaje es que ORM utilizaremos. Esta elección puede tanto simplificarnos la vida como torturarnos durante todo el proyecto si es una mala elección.

En este caso .NET nos simplifica mucho esta elección dado que solo una de las opciones es considerable que digamos, Entity Framweork. Para comenzar a comprender como funciona este ORM primero debemos observar el concepto del DBContext.

In [ ]:
public class MyContext : DbContext
{
    public DbSet<Product> Products { get; set; }
    public DbSet<Client> Clients { get; set; }
}

El context es el intermediario que nos permite interactuar con la DB, realizar querys, traer info, hacer updates y inserts.

Una vez definido el contexto básico con el cual trabajaremos, tenemos que ver como "configurar" o mapear de forma adecuada las relaciones que queremos modelar. Para esto utilizaremos FluentAPI.

Que es FluentAPI? Es un mecanismo provisto por entity framowrk el cual nos permite especificar como deben mapearse ciertos atributos / valores / relaciones del modelo que estamos creeando.

In [ ]:
public void Configure(EntityTypeBuilder<Products> builder)
{
    builder.ToTable("ProductsTableName");
    builder.Property(x => x.Name).HasMaxLength(255);
    builder.Property(x => x.type).HasDefaultValue("DefaultType");
    builder.HasMany(x => x.ListElement).WithOne().HasForeignKey(x => x.HouseholdId);
}

// Si no hacemos esto, EntityFramwork se encargara de mapearlo "segun su criterio" (lo cual no siempre es la mejor opcion)

Una vez configurado correctamente nuestro mapeo del modelo, lo siguiente que debemos poder realizar son migraciones, estas funcionan de manera muy similar a TypeORM aunque al generarse las mismas no quedan expresadas en SQL puro. Es posible realizar una migracion que ejecute un codigo RawSQL, pero las creadas por defecto con los comandos que se expresaran a continuacion son escritas en un lenguaje propio de .NET.

Agregar una migracion:

dotnet ef migrations add NameOfMigration //se le pueden agregar distintas flags --startup-project=./../Http.Api --context=Context

Aplicar migraciones: 

dotnet ef database update //se le puede indicar hasta que migracion correr o retroceder, por defecto sin indicaciones aplicara hasta la ultima sin aplicar.

In [ ]:
    public partial class AddTcjaSunsetToDefaultTaxSettings : Migration
    {
        protected override void Up(MigrationBuilder migrationBuilder)
        {
            migrationBuilder.AddColumn<bool>(
                name: "Name",
                table: "Products",
                type: "varchar(50)",
                nullable: false);

            migrationBuilder.RenameColumn(
                name: "Type",
                table: "Products",
                newName: "TypeNewName");

            migrationBuilder.Sql(@"query")

            //Existen otros metodos como AlterTable, CreateIndex entre otros aunque estos metodos se auto generan
        }

        protected override void Down(MigrationBuilder migrationBuilder)
        {
            migrationBuilder.DropColumn(
                name: "Name",
                table: "Products");

            migrationBuilder.RenameColumn(
                name: "TypeNewName",
                table: "Products",
                newName: "Type");
        }
    }

# Linq

Ya creado el modelo comenzamos a ver como interactuar con los datos, para esto tenemos Linq. 

Que es LINQ?

![Arquitectura .NET](linq-use.png)

In [ ]:

//language-level query syntax:
var linqExperts = from p in programmers
                  where p.IsNewToLINQ
                  select new LINQExpert(p);

// This is the same example using linq
var linqExperts = programmers.Where(p => p.IsNewToLINQ)
                             .Select(p => new LINQExpert(p));


Por defecto al utilizar LINQ cargamos las entidades con lazy loading, para poder "arrastrar" estas entidades a la hora de hacer una query debemos utilizar:

-Include: Incluimos un elemento directamente relacionado a la query principal.

-ThenInclude: Incluimos elementos relacionados a otros elementos incluidos.

In [ ]:
var product = await context.Products
    .Include(p => p.MetaInfo)
    .ThenInclude(m => m.Creator)
    .ThenInclude(c => c.Photo)
    .ToListAsync();

Al realizar estas querys se debe tener cuidado porque por defecto EF mantiene un traqueo sobre estos elementos y si realizamos un save changes arrastran los cambios realizados a estas entidades a la DB.

Para evitar esto podemos indicar en la query si realmente queremos que estos elementos obtenidos sean traqueados o no, para esto utilizaremos:

-AsNoTracking()

-AsTracking()

In [ ]:
// Misma query ejecutada anteriormente pero marcada para que no se traque estos elementos.
var product = await context.Products
    .Include(p => p.MetaInfo)
    .ThenInclude(m => m.Creator)
    .ThenInclude(c => c.Photo)
    .AsNoTracking()
    .ToListAsync();

// Tambien podemos marcar explicitamente que no se traque los elementos.
var product = await context.Products
    .Include(p => p.MetaInfo)
    .ThenInclude(m => m.Creator)
    .ThenInclude(c => c.Photo)
    .AsTracking()
    .ToListAsync();

Para finalizar esta vision global de EF cabe destacar que este tambien permite ejecutar querys en RawSql y realizar Transactions que podran realizar un rollback ante cualquier error detectado.

In [ ]:
// Ejemplo de transacción
var transaction = await this.context.Database.BeginTransactionAsync();
try
{
    this.Context.Products.Update(product);
    await this.context.SaveChangesAsync();
    await transaction.CommitAsync();
}           
catch (Exception ex)
{
    await transaction.RollbackAsync();
    this.Logger.LogError(ex, ex.Message);
    throw;
}

In [ ]:
FormattableString query = $@"SELECT ....";

var queryResult = await this.Context.Products.FromSqlInterpolated(query).ToListAsync();